## Kantai Bert with Byte Level Byte pair Encoding Tokenizer

1. Obtaining the dataset

### 1. Obtaining the Dataset

In [1]:
# dataset of books by Immanuel Kant

!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/master/Chapter03/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  5067k      0  0:00:02  0:00:02 --:--:-- 5068k


In [2]:
# !pip uninstall tensorflow
# !pip install transformers

In [3]:
# getting versions of transformers and tokenizers
!pip list | grep -E 'transformers|tokenizers'

tokenizers                       0.19.1
transformers                     4.42.4


In [4]:
import tokenizers
import transformers

In [5]:
%%time
# using bytelevelbpetokenizer

from pathlib import Path

paths = [str(x) for x in Path(".").glob("*.txt")]
print(paths)

tokenizer = tokenizers.ByteLevelBPETokenizer()
print(f'Tokenizer: {tokenizer}')

tokenizer.train(files=paths, vocab_size=50000, min_frequency=2, special_tokens=[
    "<start>",
    "<pad>",
    "<\s>",
    "<unknown>",
    "<mask>",
])

['kant.txt']
Tokenizer: Tokenizer(vocabulary_size=0, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)
CPU times: user 7.36 s, sys: 167 ms, total: 7.53 s
Wall time: 9.05 s


In [6]:
import os

tokenizer_dir = 'Tokenizer'

# print(os.path.exists(tokenizer_dir))
if os.path.exists(tokenizer_dir):
    # print(os.path.listdir)
    pass

if not os.path.exists(tokenizer_dir):
    os.mkdir(tokenizer_dir)
    print("Dir made")
tokenizer.save_model('Tokenizer')

Dir made


['Tokenizer/vocab.json', 'Tokenizer/merges.txt']

## Loading Tokenizers

In [7]:
from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcesssing
import tokenizers.processors
from tokenizers.processors import BertProcessing

In [8]:
tokenizer = ByteLevelBPETokenizer(
    "Tokenizer/vocab.json",
    "Tokenizer/merges.txt"
)
tokenizer

Tokenizer(vocabulary_size=19296, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [9]:
# encoding a sentence
encoding = tokenizer.encode("The thinking of the way of life")
print(f"Encoding: {encoding}")
print(f"Tokens are : {encoding.tokens}")

Encoding: Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Tokens are : ['The', 'Ġthinking', 'Ġof', 'Ġthe', 'Ġway', 'Ġof', 'Ġlife']


In [10]:
tokenizer._tokenizer.post_processor

In [11]:
tokenizer.token_to_id("<s>")

In [12]:
# fitting the BERT model
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("", tokenizer.token_to_id("")),
#     ("", tokenizer.token_to_id("")),
# )
tokenizer.enable_truncation(max_length=512)

In [13]:
encoding = tokenizer.encode("The thinking of the way of life.")
print(f"Encoding: {encoding}")
print(f"Tokens are : {encoding.tokens}")

Encoding: Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Tokens are : ['The', 'Ġthinking', 'Ġof', 'Ġthe', 'Ġway', 'Ġof', 'Ġlife', '.']


In [14]:
import torch
torch.cuda.is_available()

True

## Model Configuration

In [15]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size = 52000,
    max_position_embeddings = 514,
    num_attention_heads = 12,
    num_hidden_layers = 6,
    type_vocab_size = 1,
)
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

In [17]:
# reloading tokenizer of Roberta
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(
    "Tokenizer",
    max_length=512,
)
tokenizer

RobertaTokenizer(name_or_path='Tokenizer', vocab_size=19296, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
	19296: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	19297: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	19298: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

## Model Initialization

In [18]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [19]:
model.parameters(), model.num_parameters()

(<generator object Module.parameters at 0x7c0241c47d80>, 83504416)

In [20]:
params = list(model.parameters())
print(len(params))
print(params[0][0, :5], params[0].shape)

106
tensor([-0.0078, -0.0072,  0.0118,  0.0051,  0.0135], grad_fn=<SliceBackward0>) torch.Size([52000, 768])


In [21]:
# counting the parameters
np=0
for p in range(0,len(params)):#number of tensors
    PL2=True
    try:
        L2=len(params[p][0]) #check if 2D
    except:
        L2=1             #not 2D but 1D
        PL2=False
    L1=len(params[p])
    L3=L1*L2
    np+=L3             # number of parameters per tensor
    if PL2==True:
        print(p,L1,L2,L3)  # displaying the sizes of the parameters
    if PL2==False:
        print(p,L1,L3)  # displaying the sizes of the parameters

print(np)              # total number of parameters

0 52000 768 39936000
1 514 768 394752
2 1 768 768
3 768 768
4 768 768
5 768 768 589824
6 768 768
7 768 768 589824
8 768 768
9 768 768 589824
10 768 768
11 768 768 589824
12 768 768
13 768 768
14 768 768
15 3072 768 2359296
16 3072 3072
17 768 3072 2359296
18 768 768
19 768 768
20 768 768
21 768 768 589824
22 768 768
23 768 768 589824
24 768 768
25 768 768 589824
26 768 768
27 768 768 589824
28 768 768
29 768 768
30 768 768
31 3072 768 2359296
32 3072 3072
33 768 3072 2359296
34 768 768
35 768 768
36 768 768
37 768 768 589824
38 768 768
39 768 768 589824
40 768 768
41 768 768 589824
42 768 768
43 768 768 589824
44 768 768
45 768 768
46 768 768
47 3072 768 2359296
48 3072 3072
49 768 3072 2359296
50 768 768
51 768 768
52 768 768
53 768 768 589824
54 768 768
55 768 768 589824
56 768 768
57 768 768 589824
58 768 768
59 768 768 589824
60 768 768
61 768 768
62 768 768
63 3072 768 2359296
64 3072 3072
65 768 3072 2359296
66 768 768
67 768 768
68 768 768
69 768 768 589824
70 768 768
71 768 768

## Preparing Dataset

In [22]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "kant.txt",
    block_size = 128,
)
dataset

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 24.1 s, sys: 897 ms, total: 25 s
Wall time: 25.4 s


In [23]:
# data collator that takes sample from dataset and collates to batches
from transformers import DataCollatorForLanguageModeling

# mlm probability: % of masked tokens in training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.14,
)
data_collator

DataCollatorForLanguageModeling(tokenizer=RobertaTokenizer(name_or_path='Tokenizer', vocab_size=19296, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
	19296: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	19297: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	19298: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=True, mlm_probability=0.14

In [24]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
trainer, training_args

(<transformers.trainer.Trainer at 0x7c01ad1b8eb0>,
 TrainingArguments(
 _n_gpu=1,
 accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
 adafactor=False,
 adam_beta1=0.9,
 adam_beta2=0.999,
 adam_epsilon=1e-08,
 auto_find_batch_size=False,
 batch_eval_metrics=False,
 bf16=False,
 bf16_full_eval=False,
 data_seed=None,
 dataloader_drop_last=False,
 dataloader_num_workers=0,
 dataloader_persistent_workers=False,
 dataloader_pin_memory=True,
 dataloader_prefetch_factor=None,
 ddp_backend=None,
 ddp_broadcast_buffers=None,
 ddp_bucket_cap_mb=None,
 ddp_find_unused_parameters=None,
 ddp_timeout=1800,
 debug=[],
 deepspeed=None,
 disable_tqdm=False,
 dispatch_batches=None,
 do_eval=False,
 do_predict=False,
 do_train=False,
 eval_accumulation_steps=None,
 eval_delay=0,
 eval_do_concat_batches=True,
 eval_on_start=False,
 eval_steps=

In [25]:
%%time
trainer.train()

Step,Training Loss
500,6.591600
1000,5.679500
1500,5.132800
2000,4.814700
2500,4.609100
3000,4.403100
3500,4.263600
4000,4.150000
4500,4.050900
5000,3.965300


CPU times: user 27min 26s, sys: 9.03 s, total: 27min 36s
Wall time: 28min 27s


TrainOutput(global_step=8016, training_loss=4.413784048990337, metrics={'train_runtime': 1706.8336, 'train_samples_per_second': 300.493, 'train_steps_per_second': 4.696, 'total_flos': 2621616775994112.0, 'train_loss': 4.413784048990337, 'epoch': 3.0})

In [26]:
trainer.save_model("KantaiBERT")

In [27]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="KantaiBERT",
    tokenizer="KantaiBERT",
    device="cuda"
)

In [28]:
fill_mask("Human thinking involves human <mask>")

[{'score': 0.7162726521492004,
  'token': 18,
  'token_str': '.',
  'sequence': 'Human thinking involves human.'},
 {'score': 0.019183965399861336,
  'token': 1031,
  'token_str': ').',
  'sequence': 'Human thinking involves human).'},
 {'score': 0.01650133915245533,
  'token': 65,
  'token_str': ']',
  'sequence': 'Human thinking involves human]'},
 {'score': 0.012124789878726006,
  'token': 393,
  'token_str': ' reason',
  'sequence': 'Human thinking involves human reason'},
 {'score': 0.00953668076545,
  'token': 30,
  'token_str': ':',
  'sequence': 'Human thinking involves human:'}]

In [29]:
fill_mask("The world will remember <mask>")

[{'score': 0.7605432271957397,
  'token': 18,
  'token_str': '.',
  'sequence': 'The world will remember.'},
 {'score': 0.024912850931286812,
  'token': 1031,
  'token_str': ').',
  'sequence': 'The world will remember).'},
 {'score': 0.01722089573740959,
  'token': 35,
  'token_str': '?',
  'sequence': 'The world will remember?'},
 {'score': 0.015331648290157318,
  'token': 30,
  'token_str': ':',
  'sequence': 'The world will remember:'},
 {'score': 0.00858664233237505,
  'token': 65,
  'token_str': ']',
  'sequence': 'The world will remember]'}]

In [30]:
fill_mask("This is the general formula for the moral <mask>")

[{'score': 0.2394527643918991,
  'token': 446,
  'token_str': ' law',
  'sequence': 'This is the general formula for the moral law'},
 {'score': 0.040825679898262024,
  'token': 613,
  'token_str': ' principle',
  'sequence': 'This is the general formula for the moral principle'},
 {'score': 0.013761923648416996,
  'token': 610,
  'token_str': ' practical',
  'sequence': 'This is the general formula for the moral practical'},
 {'score': 0.01225338689982891,
  'token': 418,
  'token_str': ' conception',
  'sequence': 'This is the general formula for the moral conception'},
 {'score': 0.010023773647844791,
  'token': 600,
  'token_str': ' understanding',
  'sequence': 'This is the general formula for the moral understanding'}]